In [19]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

In [32]:
# setup
demographics_df = pd.read_csv('../data/raw/denver_demographics.csv')
crime_df = pd.read_csv('../data/raw/denver_crime.csv')
school_df = pd.read_csv('../data/raw/denver_after_school_programs.csv')
equity_df = pd.read_csv('../data/raw/denver_equity_index.csv')
neighborhoods_df = pd.read_csv('../data/raw/denver_statistical_neighborhoods_20250409.csv')
neighborhoods_df = neighborhoods_df.drop(columns=['TYPOLOGY', 'NOTES'])

## Crime

- drop feature columns
- deal with missing values
- map into neighborhoods

In [11]:
crime_df.head(5)

,OBJECTID,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,x,y
0,1,2020454617,2020454617299900,2999,0,criminal-mischief-other,public-disorder,7/26/2020 11:00:00 PM,NaN,7/28/2020 5:09:00 AM,...,-104.801111,39.797827,5,522,gateway-green-valley-ranch,1,0,1,3.196379e+06,1.716245e+06
1,2,20206013877,20206013877299900,2999,0,criminal-mischief-other,public-disorder,10/10/2020 11:55:00 AM,10/10/2020 4:59:00 AM,10/14/2020 10:20:00 PM,...,-104.905807,39.654774,3,323,hampden,1,0,1,3.167302e+06,1.663927e+06
2,3,202210816,202210816299900,2999,0,criminal-mischief-other,public-disorder,1/8/2022 7:01:00 AM,NaN,1/8/2022 7:01:00 AM,...,-105.027614,39.691784,4,421,mar-lee,1,0,1,3.132936e+06,1.677207e+06
3,4,2021159354,2021159354299900,2999,0,criminal-mischief-other,public-disorder,3/20/2021 7:38:00 AM,NaN,3/20/2021 7:38:00 AM,...,-105.025019,39.751335,1,121,jefferson-park,1,0,1,3.133553e+06,1.698903e+06
4,5,2020470873,2020470873299900,2999,0,criminal-mischief-other,public-disorder,8/4/2020 6:45:00 PM,NaN,8/4/2020 11:28:00 PM,...,-105.001965,39.740860,1,123,auraria,1,0,1,3.140056e+06,1.695122e+06


In [8]:
# summary
crime_df.describe()

,OBJECTID,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,GEO_X,GEO_Y,GEO_LON,GEO_LAT,PRECINCT_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,x,y
count,358692.000000,3.586920e+05,3.586920e+05,358692.000000,358692.000000,358692.0,358692.0,3.584390e+05,3.584390e+05,358692.000000,358692.0,358692.0,358692.000000,3.584390e+05,3.584390e+05
mean,183981.441142,6.707744e+09,6.707744e+15,2758.259147,0.235506,0.0,0.0,-1.049413e+02,3.973244e+01,385.994388,1.0,0.0,1.021043,3.157802e+06,1.694196e+06
std,105965.678882,1.347193e+10,1.347193e+16,1160.120487,0.654958,0.0,0.0,9.795205e-01,3.721786e-01,184.246986,0.0,0.0,0.247270,3.497013e+05,8.704726e+04
min,1.000000,2.020600e+04,2.020652e+10,902.000000,0.000000,0.0,0.0,-1.154641e+02,4.000000e-08,111.000000,1.0,0.0,1.000000,9.999778e-01,1.110002e+02
25%,93979.750000,2.021225e+09,2.021225e+15,2304.000000,0.000000,0.0,0.0,-1.050000e+02,3.970960e+01,222.000000,1.0,0.0,1.000000,3.140597e+06,1.683789e+06
50%,183652.500000,2.023226e+09,2.023226e+15,2399.000000,0.000000,0.0,0.0,-1.049772e+02,3.974006e+01,323.000000,1.0,0.0,1.000000,3.147018e+06,1.694869e+06
75%,275172.250000,2.020601e+10,2.020601e+16,2999.000000,0.000000,0.0,0.0,-1.049061e+02,3.976279e+01,523.000000,1.0,0.0,1.000000,3.167055e+06,1.703157e+06
max,367892.000000,2.024924e+12,2.024924e+18,7399.000000,6.000000,0.0,0.0,5.300000e-07,3.991402e+01,999.000000,1.0,0.0,33.000000,4.067477e+07,1.089045e+07


In [10]:
# all feature columns
print(crime_df.columns)

Index(['OBJECTID', 'INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE',
       'OFFENSE_CODE_EXTENSION', 'OFFENSE_TYPE_ID', 'OFFENSE_CATEGORY_ID',
       'FIRST_OCCURRENCE_DATE', 'LAST_OCCURRENCE_DATE', 'REPORTED_DATE',
       'INCIDENT_ADDRESS', 'GEO_X', 'GEO_Y', 'GEO_LON', 'GEO_LAT',
       'DISTRICT_ID', 'PRECINCT_ID', 'NEIGHBORHOOD_ID', 'IS_CRIME',
       'IS_TRAFFIC', 'VICTIM_COUNT', 'x', 'y'],
      dtype='object')


In [7]:
# list columns with missing values
for feature in crime_df.columns:
    null_sum = crime_df[feature].isna().sum()
    if null_sum > 0:
        print(f'{feature}: {null_sum} null')

LAST_OCCURRENCE_DATE: 158314 null
GEO_LON: 253 null
GEO_LAT: 253 null
DISTRICT_ID: 37 null
NEIGHBORHOOD_ID: 513 null
x: 253 null
y: 253 null


In [12]:
# dropping columns
crime_df_clean = crime_df.drop(columns=['LAST_OCCURRENCE_DATE', 'DISTRICT_ID', 'x', 'y', 'OBJECTID', 'GEO_X', 'GEO_Y', 'OFFENSE_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION'])

In [13]:
crime_df_clean.shape

(358692, 13)

In [87]:
for feature in crime_df_clean.columns:
    null_sum = crime_df_clean[feature].isna().sum()
    if null_sum > 0:
        print(f'{feature}: {null_sum} null')

neighborhood: 195 null


In [85]:
# drop rows with missing data
crime_df_clean = crime_df_clean.dropna(subset=['GEO_LON', 'GEO_LAT', 'NEIGHBORHOOD_ID'])

In [86]:
crime_df_clean.isnull().sum().sum()

195

In [34]:
# map clean df into neighborhoods based on lat/long

# Step 1: Load the neighborhoods CSV and parse WKT geometries
neighborhoods_df["the_geom"] = neighborhoods_df["the_geom"].apply(wkt.loads)
gdf_neighborhoods = gpd.GeoDataFrame(neighborhoods_df, geometry="the_geom", crs="EPSG:4326")

# Step 2: Convert your main DataFrame with points to a GeoDataFrame
crime_df_clean["geometry"] = crime_df_clean.apply(lambda row: Point(row["GEO_LON"], row["GEO_LAT"]), axis=1)
gdf_points = gpd.GeoDataFrame(crime_df_clean, geometry="geometry", crs="EPSG:4326")

# Step 3: Spatial join — find which neighborhood polygon each point is in
gdf_joined = gpd.sjoin(gdf_points, gdf_neighborhoods, how="left", predicate="within")

# Step 4: Create the 'neighborhood' column from the joined data
crime_df_clean["neighborhood"] = gdf_joined["NBHD_NAME"]

In [35]:
crime_df_clean["neighborhood"].value_counts()

neighborhood
Five Points     22191
Stapleton       18139
DIA             16823
Capitol Hill    13475
CBD             12179
                ...  
Skyland          1056
Rosedale          916
Country Club      827
Indian Creek      691
Wellshire         324
Name: count, Length: 78, dtype: int64

In [88]:
crime_df_clean = crime_df_clean.dropna(subset=['neighborhood'])
crime_df_clean.isnull().sum().sum()

0

In [89]:
crime_df_clean.head(2)

,INCIDENT_ID,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_LON,GEO_LAT,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT,geometry,neighborhood
0,2020454617,criminal-mischief-other,public-disorder,7/26/2020 11:00:00 PM,7/28/2020 5:09:00 AM,15987 E RANDOLPH PL,-104.801111,39.797827,522,gateway-green-valley-ranch,1,0,1,POINT (-104.80111124 39.7978271),Gateway - Green Valley Ranch
1,20206013877,criminal-mischief-other,public-disorder,10/10/2020 11:55:00 AM,10/14/2020 10:20:00 PM,6980 E GIRARD AVE,-104.905807,39.654774,323,hampden,1,0,1,POINT (-104.90580731 39.65477351),Hampden


In [90]:
crime_df_clean.shape

(357984, 15)

## After School Programs

- drop feature columns
- deal with missing values
- map into neighborhoods

In [38]:
school_df.head(5)

,OBJECTID,ORG_ID,SITE_ID,GLOBAL_SITEID,GLOBAL_LOCATION_TYPE,GLOBAL_LOCATION_NAME,SITE_NAME,GLOBAL_STREET_ADDR,GLOBAL_CITY,GLOBAL_STATE,...,ORG_PLIMAGE,ORG_MAIN_ZIP,GLOBAL_ADDR_SOURCE,GLOBAL_ADDR_ID,GLOBAL_LAT,GLOBAL_LONG,GLOBAL_CONTACT_PH,DPS_SCHOOL_NUM,x,y
0,1,36553,133332,124485,Recreation Center,La Alma Recreation Center,La Alma Recreation Center,1325 West 11th Ave.,Denver,CO,...,https://denvergov.org/media/denverapps/youthpr...,NaN,DAD,163468.0,39.734586,-105.004155,NaN,NaN,3.139453e+06,1.692833e+06
1,2,36553,133332,124485,Recreation Center,La Alma Recreation Center,La Alma Recreation Center,1325 West 11th Ave.,Denver,CO,...,https://denvergov.org/media/denverapps/youthpr...,NaN,DAD,163468.0,39.734586,-105.004155,NaN,NaN,3.139453e+06,1.692833e+06
2,3,36553,133332,124485,Recreation Center,La Alma Recreation Center,La Alma Recreation Center,1325 West 11th Ave.,Denver,CO,...,https://denvergov.org/media/denverapps/youthpr...,NaN,DAD,163468.0,39.734586,-105.004155,NaN,NaN,3.139453e+06,1.692833e+06
3,4,36553,133332,124485,Recreation Center,La Alma Recreation Center,La Alma Recreation Center,1325 West 11th Ave.,Denver,CO,...,https://denvergov.org/media/denverapps/youthpr...,NaN,DAD,163468.0,39.734586,-105.004155,NaN,NaN,3.139453e+06,1.692833e+06
4,5,36553,133332,124485,Recreation Center,La Alma Recreation Center,La Alma Recreation Center,1325 West 11th Ave.,Denver,CO,...,https://denvergov.org/media/denverapps/youthpr...,NaN,DAD,163468.0,39.734586,-105.004155,NaN,NaN,3.139453e+06,1.692833e+06


In [41]:
school_df_clean = school_df[['GLOBAL_LAT', 'GLOBAL_LONG', 'ORG_ID', 'GLOBAL_LOCATION_NAME', 'PROGRAM_NAME', 'PROGRAM_STATUS', 'PROGRAM_FEE', 'ORG_NAME', 'ORG_WEB_ADDR']]
school_df_clean.shape

(27379, 9)

In [42]:
school_df_clean.isnull().sum()

GLOBAL_LAT               95
GLOBAL_LONG              95
ORG_ID                    0
GLOBAL_LOCATION_NAME     95
PROGRAM_NAME            996
PROGRAM_STATUS          996
PROGRAM_FEE             996
ORG_NAME                  9
ORG_WEB_ADDR             65
dtype: int64

In [43]:
# drop missing values from important rows - lat/lng
school_df_clean = school_df_clean.dropna(subset=['GLOBAL_LONG', 'GLOBAL_LAT'])

In [46]:
# map clean df into neighborhoods based on lat/long

# Step 2: Convert your main DataFrame with points to a GeoDataFrame
school_df_clean["geometry"] = school_df_clean.apply(lambda row: Point(row["GLOBAL_LONG"], row["GLOBAL_LAT"]), axis=1)
school_gdf_points = gpd.GeoDataFrame(school_df_clean, geometry="geometry", crs="EPSG:4326")

# Step 3: Spatial join — find which neighborhood polygon each point is in
school_gdf_joined = gpd.sjoin(school_gdf_points, gdf_neighborhoods, how="left", predicate="within")

# Step 4: Create the 'neighborhood' column from the joined data
school_df_clean["neighborhood"] = school_gdf_joined["NBHD_NAME"]

In [47]:
school_df_clean["neighborhood"].value_counts()

neighborhood
Gateway - Green Valley Ranch    1656
Stapleton                       1636
Montbello                       1353
Mar Lee                          949
Hilltop                          751
                                ... 
Southmoor Park                    19
Goldsmith                         16
Kennedy                           13
Auraria                           13
Union Station                      7
Name: count, Length: 76, dtype: int64

In [48]:
school_df_clean.isnull().sum()

GLOBAL_LAT                 0
GLOBAL_LONG                0
ORG_ID                     0
GLOBAL_LOCATION_NAME       0
PROGRAM_NAME             988
PROGRAM_STATUS           988
PROGRAM_FEE              988
ORG_NAME                   9
ORG_WEB_ADDR              65
geometry                   0
neighborhood            2212
dtype: int64

In [50]:
school_df_clean = school_df_clean[school_df_clean["GLOBAL_LOCATION_NAME"] != "Virtual"]

In [54]:
school_df_clean = school_df_clean.dropna(subset=['neighborhood'])

In [55]:
school_df_clean.isnull().sum()

GLOBAL_LAT                0
GLOBAL_LONG               0
ORG_ID                    0
GLOBAL_LOCATION_NAME      0
PROGRAM_NAME            768
PROGRAM_STATUS          768
PROGRAM_FEE             768
ORG_NAME                  6
ORG_WEB_ADDR             57
geometry                  0
neighborhood              0
dtype: int64

In [56]:
school_df_clean.shape

(25072, 11)

In [57]:
school_df_clean.head(5)

,GLOBAL_LAT,GLOBAL_LONG,ORG_ID,GLOBAL_LOCATION_NAME,PROGRAM_NAME,PROGRAM_STATUS,PROGRAM_FEE,ORG_NAME,ORG_WEB_ADDR,geometry,neighborhood
0,39.734586,-105.004155,36553,La Alma Recreation Center,Indoor Pool,Active,1.0,Denver Parks and Recreation - City & County of...,https://www.denvergov.org/recreation,POINT (-105.0041546 39.73458561),Lincoln Park
1,39.734586,-105.004155,36553,La Alma Recreation Center,Indoor Pool,Active,1.0,Denver Parks and Recreation - City & County of...,https://www.denvergov.org/recreation,POINT (-105.0041546 39.73458561),Lincoln Park
2,39.734586,-105.004155,36553,La Alma Recreation Center,Indoor Pool,Active,1.0,Denver Parks and Recreation - City & County of...,https://www.denvergov.org/recreation,POINT (-105.0041546 39.73458561),Lincoln Park
3,39.734586,-105.004155,36553,La Alma Recreation Center,Indoor Pool,Active,1.0,Denver Parks and Recreation - City & County of...,https://www.denvergov.org/recreation,POINT (-105.0041546 39.73458561),Lincoln Park
4,39.734586,-105.004155,36553,La Alma Recreation Center,Indoor Pool,Active,1.0,Denver Parks and Recreation - City & County of...,https://www.denvergov.org/recreation,POINT (-105.0041546 39.73458561),Lincoln Park


## Neighborhood Data

In [70]:
# finding mismatched names
equity_nhbds = equity_df["NBHD_NAME"]
demo_nhbds = demographics_df["Neighborhood Name"]
print(set(equity_nhbds) - set(demo_nhbds))
print(set(demo_nhbds) - set(equity_nhbds))

{'Stapleton'}
{'Central Park'}


In [71]:
# changing equity_df Stapleton to Central Park
equity_df["NBHD_NAME"] = equity_df["NBHD_NAME"].replace({'Stapleton': 'Central Park'})

In [73]:
demographics_df.shape

(78, 151)

In [74]:
# merging equity and demographics dataframes so in single df
merged_df = pd.merge(equity_df, demographics_df, left_on="NBHD_NAME", right_on="Neighborhood Name", how="inner")
merged_df.shape

(78, 162)

In [78]:
merged_df = merged_df.drop(columns=["NBHD_NAME"])
merged_df.shape

(78, 161)

In [93]:
# test mismatched names with other dataframes - both crime and school programs has same neighborhood name values
merged_nhbds = merged_df["Neighborhood Name"]
crime_nhbds = crime_df_clean["neighborhood"]
school_nhbds = school_df_clean["neighborhood"]
print(set(crime_nhbds) - set(merged_nhbds))
print(set(merged_nhbds) - set(crime_nhbds))

{'Stapleton'}
{'Central Park'}


In [94]:
print(set(school_nhbds) - set(merged_nhbds))
print(set(merged_nhbds) - set(school_nhbds))

{'Stapleton'}
{'Windsor', 'Country Club', 'Central Park'}


In [96]:
# changing Stapleton to Central Park
crime_df_clean["neighborhood"] = crime_df_clean["neighborhood"].replace({'Stapleton': 'Central Park'})
school_df_clean["neighborhood"] = school_df_clean["neighborhood"].replace({'Stapleton': 'Central Park'})
print(crime_df_clean.shape)
print(school_df_clean.shape)

(357984, 15)
(25072, 11)
